In [ ]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

import keras
from keras.utils import to_categorical
from keras.constraints import maxnorm
from keras.models import Sequential, load_model
from keras.optimizers import SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, PReLU, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

In [ ]:
def LoadData():
    train = pd.read_csv('./data/train.csv')
    test = pd.read_csv('./data/test.csv')
    Y_train = np.array(train['label'])
    Y_train = to_categorical(Y_train)
    
    X = np.array(train['feature'])
    X_train = []
    for i in range(X.shape[0]):
        x = np.array(X[i].split(' '))
        x = x.astype(np.int)
        X_train.append(x)
    X_train = np.array(X_train)
    X_train = X_train.reshape(X_train.shape[0],48,48,1)
    
    X = np.array(test['feature'])
    X_test = []
    for i in range(X.shape[0]):
        x = np.array(X[i].split(' '))
        x = x.astype(np.int)
        X_test.append(x)
    X_test = np.array(X_test)
    X_test = X_test.reshape(X_test.shape[0],48,48,1)
    
    return X_train, Y_train, X_test

In [ ]:
labels = {0: "angry", 1:"disgust", 2:"fear", 3:"happy", 4:"sad", 5:"surprise", 6:"neutral"}
X_train, Y_train, X_test = LoadData()

In [ ]:
def model_create():
    model = Sequential()
    model.add(Conv2D(512, (3, 3), activation='relu', input_shape=(48,48,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.2))

    for i in range(3):
        model.add(Conv2D(512, (3, 3), activation='relu', kernel_constraint=maxnorm(5)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Dropout(0.5))      
        model.add(BatchNormalization(scale=True, center=False, momentum=0.99, epsilon=0.001))

    model.add(Flatten()) 
    
    for i in range(2):
        model.add(Dense(128, activation='relu'))
#         model.add(Dropout(0.5))

    model.add(Dense(7, activation='softmax'))
    prop = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    model.compile(optimizer=prop,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
outfile  = 'model15'
model = model_create()
checkpoints = ModelCheckpoint('./model/' + outfile  + '.h5', verbose=2, 
                              monitor='val_acc', save_best_only=True, mode='auto')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, 
                              verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
trainX, testX, trainY, testY = train_test_split(X_train,Y_train,test_size= 0.1)

In [ ]:
BS = 50
EPOCHS = 100

# train the network
train = model.fit_generator(datagen.flow(trainX, trainY, batch_size = BS),
                            validation_data=(testX, testY), 
                            epochs=EPOCHS,
                            callbacks=[reduce_lr]
                            callbacks=[checkpoints,reduce_lr])

In [ ]:
train = model.fit(X_train, Y_train, batch_size=256, validation_split=0.2,
                  epochs=150, verbose=1,shuffle=True,callbacks=[checkpoints,reduce_lr])

In [ ]:
max(train.history['val_acc'])

In [ ]:
model = load_model('./model/' + outfile  + '.h5')
ans = model.predict_classes(X_test).astype('int')
ans = np.column_stack([list(range(X_test.shape[0])), ans])
ans = pd.DataFrame(ans,columns=['id', 'label'])
ans.to_csv('data/'+ outfile +'.csv', index=False)

In [ ]:
def plot_sample(X):
    plt.imshow(X.reshape(48,48),cmap="gray")

In [ ]:
plot_sample(X_train[2])